In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json

Brief intro into the bpps files (base pairing probability matrix of every possible base pair). These are 107 X 107 symmetric matrices, 107 is the number of bases in a RNA sequence

In [2]:
data = np.load('/kaggle/input/stanford-covid-vaccine/bpps/id_001f94081.npy')
print(data.shape)
print(data)

(107, 107)
[[0.         0.         0.         ... 0.         0.         0.0146736 ]
 [0.         0.         0.         ... 0.         0.         0.00620374]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.0146736  0.00620374 0.         ... 0.         0.         0.        ]]


A detailed look at the bpps data

In [3]:
pd.set_option('display.max_colwidth', None)
bpps_data_list = list(data)
bpps_data_list_68 = bpps_data_list[0:68]
bpps_data_list_68_68 = []

for i in range(len(bpps_data_list_68)):
    bpps_data_list_68_68.append(bpps_data_list_68[i][0:68])

print(bpps_data_list_68_68[0])
print(bpps_data_list_68_68[0][7])
print(bpps_data_list_68_68[1])

[0.         0.         0.         0.         0.         0.
 0.         0.00655413 0.00922102 0.00809437 0.0217857  0.
 0.         0.         0.         0.         0.00112002 0.
 0.         0.         0.         0.         0.         0.00465301
 0.00241507 0.         0.         0.         0.         0.0114696
 0.00492862 0.         0.00353196 0.         0.         0.
 0.         0.         0.         0.00159406 0.00629747 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.0052959  0.00605378 0.
 0.         0.         0.0012753  0.0136668  0.         0.
 0.00152313 0.        ]
0.00655413
[0.         0.         0.         0.         0.         0.
 0.         0.0182053  0.00511209 0.0386527  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.0043234
 0.         0.         0.         0.         0.         0.

Ingesting the test dataset which is in json format, then bringing it in pandas dataframe

In [4]:
pd.set_option('display.max_colwidth', None)
json_data = [json.loads(line) for line in open('/kaggle/input/stanford-covid-vaccine/test.json', 'r')]
test_df = pd.DataFrame(json_data)
print(test_df.shape)
test_df.head(2).T

(3634, 7)


,0,1
index,0,1
id,id_00073f8be,id_000ae4237
sequence,GGAAAAGUACGACUUGAGUACGGAAAACGUACCAACUCGAUUAAAAUGGUCAAAGAGGUCGAAAUACAGAUGACCUUCGGGUUAUCAAAAGAAACAACAACAACAAC,GGAAACGGGUUCCGCGGAUUGCUGCUAAUAAGAGUAAUCUCUAAAUGCAGCUACCGGCUCUAUAAUGAGCAAAAACGGUAAAUCCCGACAAGCUUGAUUUCGAUCAAGCAAAAGAAACAACAACAACAAC
structure,......((((((((((.(((((.....))))))))((((((((...)))))...)))))))...))).(((((((....))))))).....................,.....((((..((((((...(((((.....((((....))))....)))))..)))((((......)))).....))).....))))....(((((((....))))))).....................
predicted_loop_type,EEEEEESSSSSSSSSSBSSSSSHHHHHSSSSSSSSSSSSSSSSHHHSSSSSBBBSSSSSSSBBBSSSXSSSSSSSHHHHSSSSSSSEEEEEEEEEEEEEEEEEEEEE,EEEEESSSSIISSSSSSIIISSSSSIIIIISSSSHHHHSSSSIIIISSSSSIISSSSSSSHHHHHHSSSSMMMMMSSSIIIIISSSSXXXXSSSSSSSHHHHSSSSSSSEEEEEEEEEEEEEEEEEEEEE
seq_length,107,130
seq_scored,68,91


Ingesting the training dataset which is in json format, then bringing it in pandas dataframe

In [5]:
pd.set_option('display.max_colwidth', None)
json_data = [json.loads(line) for line in open('/kaggle/input/stanford-covid-vaccine/train.json', 'r')]
train_df = pd.DataFrame(json_data)
print(train_df.shape)
train_df.head(2).T

(2400, 19)


,0,1
index,0,1
id,id_001f94081,id_0049f53ba
sequence,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC,GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACCGCGCGCGCUUAUGCAAGUUGCCCGCGGCGUUCGCGCUGUGAAAAGAAACAACAACAACAAC
structure,.....((((((.......)))).)).((.....((..((((((....))))))..)).....))....(((((((....))))))).....................,.....(((((((((((((((((((((((....)))))))))).)))))))))))))..(((...))).(((((((....))))))).....................
predicted_loop_type,EEEEESSSSSSHHHHHHHSSSSBSSXSSIIIIISSIISSSSSSHHHHSSSSSSIISSIIIIISSXXXXSSSSSSSHHHHSSSSSSSEEEEEEEEEEEEEEEEEEEEE,EEEEESSSSSSSSSSSSSSSSSSSSSSSHHHHSSSSSSSSSSBSSSSSSSSSSSSSMMSSSHHHSSSMSSSSSSSHHHHSSSSSSSEEEEEEEEEEEEEEEEEEEEE
signal_to_noise,6.894,0.193
SN_filter,1.0,0.0
seq_length,107,107
seq_scored,68,68
reactivity_error,"[0.1359, 0.207, 0.1633, 0.1452, 0.1314, 0.105, 0.0821, 0.0964, 0.0756, 0.1087, 0.1377, 0.1544, 0.1622, 0.1388, 0.1284, 0.1009, 0.0941, 0.0564, 0.0417, 0.0596, 0.0482, 0.1041, 0.0942, 0.052, 0.0583, 0.0403, 0.0491, 0.1003, 0.0525, 0.081, 0.1103, 0.0707, 0.0797, 0.0997, 0.0968, 0.0939, 0.0931, 0.0604, 0.0427, 0.0331, 0.0412, 0.0286, 0.0415, 0.0394, 0.0636, 0.0816, 0.0474, 0.0295, 0.0163, 0.0211, 0.0426, 0.0592, 0.0778, 0.0579, 0.0606, 0.0462, 0.0354, 0.0324, 0.0351, 0.0539, 0.0565, 0.0489, 0.0221, 0.0254, 0.0328, 0.0394, 0.0409, 0.0547]","[2.8272, 2.8272, 2.8272, 4.7343, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 2.5676, 4.2902, 2.3644, 2.3644, 2.3644, 2.3644, 2.3644, 2.3644, 3.0009, 2.3356, 2.923, 4.4649, 4.0411, 2.6232, 2.4985, 2.3879, 1.467, 1.467, 1.467, 1.467, 1.467, 1.467, 2.2892, 1.4271, 2.2008, 1.3919, 1.3919, 1.3919, 2.1211, 2.1008, 2.028, 1.3001, 1.3001, 1.3001, 1.962, 1.9018, 1.8467, 1.2307, 2.0146, 1.7072, 1.6677, 1.165, 2.0515, 1.7922, 1.5873, 1.5375, 1.4931, 1.3286, 2.273, 0.976, 0.976]"


Bringing the bpps data of every sample

Vectorized method, without creating another dataframe, thereby saving memory

In [6]:
def get_bpps(row):
    file_location = "".join(['/kaggle/input/stanford-covid-vaccine/bpps/',row['id'],'.npy'])
    bpps_data = np.load(file_location)
    bpps_data_list = list(bpps_data)
    bpps_data_list_68 = bpps_data_list[0:68]
    bpps_data_list_68_68 = []
    for j in range(len(bpps_data_list_68)):
        bpps_data_list_68_68.append(bpps_data_list_68[j][0:68])
        
    row['bpps'] = bpps_data_list_68_68
    return row['bpps']

train_df['bpps'] = train_df.apply(get_bpps, axis=1)

## Verifications
print(train_df.shape)
print(len(train_df['sequence'][100]))
print(len(train_df['deg_50C']))
print(len(train_df['deg_error_50C'][4]))
print(len(train_df['bpps']))
print(len(train_df['bpps'][2399]))
print(len(train_df['bpps'][2399][60]))
print(len(train_df['deg_error_Mg_50C'][1]))
print((train_df['reactivity'][1]))
print((train_df['sequence'][1]))
print((train_df['structure'][1]))

(2400, 20)
107
2400
68
2400
68
68
68
[0.0, 0.0, 0.0, 2.2965, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0668, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0878, 0.0, -1.0334, 4.7697, 2.8911, 1.2158, 1.1483, 1.0878, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0334, 0.0, 0.9842, 0.0, 0.0, 0.0, 0.9395, -0.9395, 0.8986, 0.0, 0.0, 0.0, 0.8612, 0.8267, 0.795, 0.0, 1.4763, 0.7127, 0.689, 0.0, 2.4316, 1.6758, 1.0599, 1.0082, 0.9613, 0.4697, -1.2631, 0.0, 0.0]
GGAAAAAGCGCGCGCGGUUAGCGCGCGCUUUUGCGCGCGCUGUACCGCGCGCGCUUAUGCAAGUUGCCCGCGGCGUUCGCGCUGUGAAAAGAAACAACAACAACAAC
.....(((((((((((((((((((((((....)))))))))).)))))))))))))..(((...))).(((((((....))))))).....................


List based approach, takes way much shorter time than the brute force method

In [7]:
colms = ['id_base', 'sequence', 'structure', 'predicted_loop_type', 'signal_to_noise', 'SN_filter', 'reactivity_error'
         , 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C', 'reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C', 'bpps']

def create_rows_per_base(col_name):
    col_list = []
    for i in range(train_df.shape[0]):
        if col_name == 'id_base':
            s = train_df['id'][i]
            for j in range(68):
                col_list.append("_".join([s, str(j)]))
        else:
            for j in range(68):
                if (col_name == 'signal_to_noise') or (col_name == 'SN_filter'):
                    col_list.append(train_df[col_name][i])
                else:
                    col_list.append(train_df[col_name][i][j])
            
    return col_list

d = {}
for colum in colms:
    d[colum] = create_rows_per_base(colum)

seqn_train_df = pd.DataFrame(d, columns=colms)
seqn_train_df['paired_or_unpaired'] = seqn_train_df['structure'].apply(lambda x: 1 if x == '(' else 1 if x == ')' else 0)

In [8]:
seqn_train_df[['id_base','sequence','reactivity','deg_Mg_50C']].head(10)

,id_base,sequence,reactivity,deg_Mg_50C
0,id_001f94081_0,G,0.3297,0.3581
1,id_001f94081_1,G,1.5693,2.9683
2,id_001f94081_2,A,1.1227,0.2589
3,id_001f94081_3,A,0.8686,1.4552
4,id_001f94081_4,A,0.7217,0.7244
5,id_001f94081_5,A,0.4384,0.4971
6,id_001f94081_6,G,0.2560,0.3819
7,id_001f94081_7,C,0.3364,0.9115
8,id_001f94081_8,U,0.2168,0.1668
9,id_001f94081_9,C,0.3583,1.0613


In [9]:
train_df[['id','sequence','reactivity','deg_Mg_50C']][train_df['id']=='id_001f94081'].head(10)

,id,sequence,reactivity,deg_Mg_50C
0,id_001f94081,GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC,"[0.3297, 1.5693, 1.1227, 0.8686, 0.7217, 0.4384, 0.256, 0.3364, 0.2168, 0.3583, 0.9541, 1.4113, 1.6911, 1.2494, 1.1895, 0.6909, 0.4736, 0.1754, 0.0582, 0.2173, 0.0785, 0.8249, 0.7638, 0.1095, 0.2568, 0.0895, 0.1576, 0.7727, 0.1573, 0.5043, 1.0444, 0.4766, 0.5588, 0.9054, 1.0125, 1.0482, 1.044, 0.4522, 0.211, 0.0461, 0.082, 0.0643, 0.1526, 0.0894, 0.5081, 1.0745, 0.3215, 0.0716, 0.0244, 0.0123, 0.1984, 0.4961, 1.0641, 0.6394, 0.6789, 0.365, 0.1741, 0.1408, 0.1646, 0.5389, 0.683, 0.4273, 0.0527, 0.0693, 0.1398, 0.2937, 0.2362, 0.5731]","[0.3581, 2.9683, 0.2589, 1.4552, 0.7244, 0.4971, 0.3819, 0.9115, 0.1668, 1.0613, 1.1317, 0.7468, 1.9049, 1.103, 0.7317, 0.7649, 0.701, 0.4327, 0.236, 0.4039, 0.0373, 0.3362, 0.2914, 0.3956, 0.3074, 0.3079, 0.2379, 0.2027, 0.3721, 0.4504, 0.6303, 0.2092, 0.6219, 1.0619, 0.8667, 0.9206, 0.8516, 0.6308, 0.1756, 0.4022, 0.2236, 0.1452, 0.1469, 0.6622, 0.5692, 0.4583, 0.2705, 0.2798, 0.0298, 0.0367, 0.2299, 0.2873, 1.7464, 0.3233, 0.2996, 0.2029, 0.2311, 0.2032, 0.4079, 0.5544, 0.3829, 0.3445, 0.1898, 0.2649, 0.2714, 0.4812, 0.7026, 0.4254]"


Creating 68 BPPS columns per base

In [10]:
def add_bpps_per_base(s,j):
    return s[j]

vect_func = np.vectorize(add_bpps_per_base, excluded=['j'])

for jj in range(68):
    seqn_train_df["_".join(['bpps',str(jj)])] = vect_func(seqn_train_df['bpps'], jj)

In [11]:
seqn_train_df.shape

(163200, 86)

In [12]:
seqn_train_df.columns

Index(['id_base', 'sequence', 'structure', 'predicted_loop_type',
       'signal_to_noise', 'SN_filter', 'reactivity_error', 'deg_error_Mg_pH10',
       'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C', 'reactivity',
       'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C', 'bpps',
       'paired_or_unpaired', 'bpps_0', 'bpps_1', 'bpps_2', 'bpps_3', 'bpps_4',
       'bpps_5', 'bpps_6', 'bpps_7', 'bpps_8', 'bpps_9', 'bpps_10', 'bpps_11',
       'bpps_12', 'bpps_13', 'bpps_14', 'bpps_15', 'bpps_16', 'bpps_17',
       'bpps_18', 'bpps_19', 'bpps_20', 'bpps_21', 'bpps_22', 'bpps_23',
       'bpps_24', 'bpps_25', 'bpps_26', 'bpps_27', 'bpps_28', 'bpps_29',
       'bpps_30', 'bpps_31', 'bpps_32', 'bpps_33', 'bpps_34', 'bpps_35',
       'bpps_36', 'bpps_37', 'bpps_38', 'bpps_39', 'bpps_40', 'bpps_41',
       'bpps_42', 'bpps_43', 'bpps_44', 'bpps_45', 'bpps_46', 'bpps_47',
       'bpps_48', 'bpps_49', 'bpps_50', 'bpps_51', 'bpps_52', 'bpps_53',
       'bpps_54', 'bpps_55', 'bpps_56', 'b

In [13]:
seqn_train_df[['id_base','predicted_loop_type','reactivity','bpps_0','bpps_1','bpps_7']].head()

,id_base,predicted_loop_type,reactivity,bpps_0,bpps_1,bpps_7
0,id_001f94081_0,E,0.3297,0.0,0.0,0.006554
1,id_001f94081_1,E,1.5693,0.0,0.0,0.018205
2,id_001f94081_2,E,1.1227,0.0,0.0,0.000000
3,id_001f94081_3,E,0.8686,0.0,0.0,0.000000
4,id_001f94081_4,E,0.7217,0.0,0.0,0.000000


In [14]:
## Verifications
print(seqn_train_df.shape)
print(len(seqn_train_df['sequence'][100]))
print(len(seqn_train_df['deg_50C']))
print((seqn_train_df['deg_error_50C'][4]))
print(len(seqn_train_df['bpps']))
print(len(seqn_train_df['bpps'][2399]))
print((seqn_train_df['bpps'][2399][60]))
print((seqn_train_df['deg_error_Mg_50C'][1]))
print((seqn_train_df['reactivity'][1]))
print((seqn_train_df['sequence'][1]))
print((seqn_train_df['structure'][1]))

(163200, 86)
1
163200
0.1703
163200
68
0.0
0.275
1.5693
G
.


In [15]:
predicted_loop_type_categories = seqn_train_df[['predicted_loop_type']]
predicted_loop_type_categories.head(10)

,predicted_loop_type
0,E
1,E
2,E
3,E
4,E
5,S
6,S
7,S
8,S
9,S


In [16]:
from sklearn.preprocessing import OneHotEncoder
loop_type_cat_encoder = OneHotEncoder()

predicted_loop_type_cat_1hot = loop_type_cat_encoder.fit_transform(predicted_loop_type_categories)
predicted_loop_type_cat_1hot.toarray()

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [17]:
loop_type_cat_encoder.categories_

[array(['B', 'E', 'H', 'I', 'M', 'S', 'X'], dtype=object)]